# Day 1 - Lab 1: AI-Powered Requirements & User Stories

**Objective:** Use a Large Language Model (LLM) to decompose a vague problem statement into structured features, user personas, and Agile user stories, culminating in a machine-readable JSON artifact.

**Estimated Time:** 90 minutes

**Introduction:**
Welcome to the first hands-on lab of the AI-Driven Software Engineering Program! All great software projects begin with a clear understanding of the problem to be solved. In this lab, you will take on the role of a tech lead or product manager and use an LLM as a co-pilot to transform a simple, high-level problem into a set of well-defined, actionable requirements. This process is fundamental to ensuring that the team builds the *right* product.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM).

**Model Selection:**
Our `utils.py` script is configured to work with multiple AI providers. You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in the `RECOMMENDED_MODELS` dictionary in `utils.py`. For example, to use a Hugging Face model, you could change the line to: `client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-3.3-70B-Instruct")`

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client for our chosen LLM.
- `get_completion()`: To send a prompt to the LLM and get a response.
- `save_artifact()`: To save our generated requirements to a file.

In [18]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-flash")

2025-10-27 16:20:43,328 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-flash latency_ms=None artifacts_path=None


## Step 2: The Problem Statement

Every project starts with a problem. Our problem is a common one in many organizations:

> **"We need a tool to help our company's new hires get up to speed."**

This statement is intentionally vague. Our job is to use the LLM to add structure and detail to it.

In [19]:
problem_statement = "We need a tool to help our company's new hires get up to speed."

## Step 3: The Challenges

Complete the following challenges in order. Each one builds upon the last, increasing in technical complexity and value.

### Challenge 1 (Foundational): Brainstorming Features

**Task:** Use the LLM to brainstorm a list of potential features and user personas based on the problem statement.

**Instructions:**
1. Write a simple prompt that asks the LLM to brainstorm features for the onboarding tool.
2. Write a second prompt to identify three distinct user personas who would use this tool.
3. Run both prompts and review the markdown output.

**Expected Quality:** The output should be a simple, readable markdown list of features and a description of the personas. This is a good first step but lacks the structure needed for automation.

In [24]:
# This prompt should ask the LLM to brainstorm features based on the problem_statement.
features_prompt = f"""Based on the problem statement: {problem_statement}, brainstorm a list of useful features for a new hire onboarding tool. 
The new hires are a mix of recent graduates and experienced professionals joining various departments. Ensure that the onboarding tool can cater to different learning styles and paces.
Consider both the needs of new employees and HR or team managers. Include ideas that improve engagement, make the process smoother, and help new hires ramp up faster. 
Organize the ideas by category (e.g., communication, learning, administration, culture). Provide a brief description for each feature. Ensure that the new hire onboarding tool includes learning models that embody our mission statement: Empowering people to change the world. Format the result in a markdown text format."""

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# This prompt should ask the LLM to identify three user personas based on the problem_statement.
personas_prompt = f"""Based on the problem statement: {problem_statement} and the following brainstormed features: {brainstormed_features}, identify and describe three distinct user personas who would interact with this tool. For each persona, describe their role and main goal. Include a mix of new hires and HR or team managers. Format the result in a markdown text format."""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)

--- Brainstorming Features ---
Here's a list of useful features for a new hire onboarding tool, categorized and designed to meet the specified requirements, including the mission statement: "Empowering people to change the world."

---

## New Hire Onboarding Tool Features

### I. Learning & Development

Features focused on knowledge acquisition and skill-building, catering to diverse learning styles and paces.

*   **Personalized Learning Paths:**
    *   **Description:** Dynamic learning tracks tailored to the new hire's role, department, prior experience (e.g., recent grad vs. experienced professional), and identified learning style. Includes recommended modules, resources, and deadlines.
*   **Interactive Learning Modules:**
    *   **Description:** Engaging, multimedia-rich content (videos, animations, quizzes, simulations, drag-and-drop exercises) covering company policies, tools, security protocols, and foundational knowledge. Allows for self-paced learning.
*   **Comprehensive 

### Challenge 2 (Intermediate): Generating Formal User Stories

**Task:** Now, let's increase the value by generating structured, formal Agile User Stories.

**Instructions:**
1. Create a new, more sophisticated prompt.
2. This prompt should instruct the LLM to act as a Senior Product Manager.
3. It must use the brainstormed features and personas from the previous step as context.
4. The key instruction is to generate a list of user stories, each with detailed acceptance criteria in Gherkin format (`Given/When/Then`).
5. **Crucially, the prompt must demand the final output be a well-formed JSON array of objects.** Each object should represent a user story and have keys like `id`, `user_story`, `persona`, and `acceptance_criteria`.

> **Tip:** If the LLM's output isn't perfect JSON, try making your prompt even more specific. You can tell it, 'Do not include any text before or after the JSON array. Your response must begin with [ and end with ].'

**Expected Quality:** The output should not be markdown, but a clean, parsable JSON string. This is a significant step up in value, as a JSON artifact can be automatically processed by other systems (e.g., imported into Jira).

In [27]:
# This prompt needs to instruct the LLM to act as a Senior Product Manager and convert the
# brainstormed features and personas into a structured JSON array of user stories.
# Tip: Be very specific about the required JSON format in your prompt instructions. Tell it what keys to use and what the data types should be.
json_user_stories_prompt = f""" 
Act as a Senior Product Manager for a software development team. Based on the following problem statement: {problem_statement}, the brainstormed features: {brainstormed_features}, and the identified user personas: {user_personas}, create a structured JSON array of user stories. Each user story should be a JSON object with the following keys:
- "id": A unique integer identifier for the user story.
- "user_story": A string formatted as "As a [persona], I want [feature] so that [benefit]."
- "persona": A string indicating which user persona the story is for.
- "acceptance_criteria": An array of strings, each describing a specific condition that must be met for the story to be considered complete. These criteria should follow the Gherkin format (e.g., Given [context], When [action], Then [outcome]).
Ensure the output is valid JSON and follows this exact structure. Provide at least five user stories covering different personas and features.

Here is an example of the expected JSON format for one user story:
    "id": 1,
    "user_story": "As a new hire, I want an interactive onboarding checklist so that I can track my progress through the onboarding process.",
    "persona": "New Hire",
    "acceptance_criteria": [
        "Given I am a new hire, When I access the onboarding tool, Then I should see an interactive checklist of onboarding tasks.",
        "When I complete a task, Then it should be marked as completed on the checklist.",
        "Then I should receive a notification when all tasks are completed."
    ]
"""

print("--- Generating User Stories as JSON ---")
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Let's try to parse the JSON to see if the LLM followed instructions
try:
    # The LLM might wrap the JSON in markdown fences (```json ... ```).
    # We'll clean that up before parsing.
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    
    if user_stories_json:
        print("\n--- Sample User Story ---")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = []

--- Generating User Stories as JSON ---
Successfully parsed LLM output as JSON.

--- Sample User Story ---
{
  "id": 1,
  "user_story": "As Amelia, the Aspiring Software Engineer, I want a personalized learning path tailored to my role so that I can efficiently acquire the necessary skills and knowledge.",
  "persona": "Amelia, the Aspiring Software Engineer",
  "acceptance_criteria": [
    "Given I am a new Software Engineer, When I log into the onboarding tool, Then I should see a learning path specifically designed for my role.",
    "Given my prior experience as a recent grad, When I view my learning path, Then it should recommend foundational modules relevant to my background.",
    "When I complete a module, Then my progress should be updated on my personalized learning path."
  ]
}


### Challenge 3 (Advanced): Programmatic Validation and Artifact Creation

**Task:** Now for the highest-value step. Instead of just looking at the JSON, we will programmatically validate it and save it as a formal project artifact. This ensures reliability and prepares the requirements for automated use in later stages of the SDLC.

**Instructions:**
1. Complete the `validate_and_save_stories` function below.
2. The function should iterate through the list of stories.
3. For each story, it must validate that the required keys are present and that the acceptance criteria list is not empty.
4. If all stories are valid, it should save the data to `artifacts/day1_user_stories.json`.

**Expected Quality:** A robust script that guarantees the integrity of our requirements artifact. The final output is a validated `day1_user_stories.json` file in the `artifacts` directory, ready to be used as a reliable input for Day 2.

In [31]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid.
    Returns True if validation and saving were successful, False otherwise.
    """
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    # TODO: Implement the validation logic inside this function.
    # 1. Loop through each story in the 'stories_data' list.
    # 2. For each story, check if it contains all the 'required_keys'.
    # 3. Also check if the 'acceptance_criteria' list is not empty.
    # 4. If a story is invalid, print an error message and set 'all_stories_valid' to False.
    #    (You can use 'continue' to skip to the next story).

    for story in stories_data:
        # Check if all required keys are present
        if not all(key in story for key in required_keys):
            print(f"Validation Failed: Story with id {story.get('id', 'Unknown')} is missing required keys.")
            all_stories_valid = False
            continue
        
        # Check if acceptance_criteria is a non-empty list
        if not isinstance(story['acceptance_criteria'], list) or not story['acceptance_criteria'] or len(story['acceptance_criteria']) == 0:
            print(f"Validation Failed: Story with id {story['id']} has invalid or empty acceptance_criteria.")
            all_stories_valid = False
            continue

    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/day1_user_stories.json"
        
        # TODO: Call the save_artifact function from utils.py to save the data.
        # Remember to convert the Python list back to a JSON string using json.dumps().

        save_artifact(json.dumps(stories_data, indent=2), artifact_path, overwrite=True)
        print(f"User stories saved to {artifact_path}.")
        return True
        
    else:
        print("\nValidation failed. Artifact not saved.")
        return False

# Run the validation on the JSON data from the previous step
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or not defined.")


All user stories passed validation.
User stories saved to artifacts/day1_user_stories.json.


## Lab Conclusion

Congratulations! You have completed the first lab. You started with a vague, one-sentence problem and finished with a structured, validated, machine-readable requirements artifact. This is the critical first step in an AI-assisted software development lifecycle. The `day1_user_stories.json` file you created will be the direct input for our next lab, where we will generate a formal Product Requirements Document (PRD).

> **Key Takeaway:** The single most important skill demonstrated in this lab is turning unstructured ideas into structured, machine-readable data (JSON). This transformation is what enables automation and integration with other tools later in the SDLC.